## Installing Pytorch

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x5808c000 @  0x7f110bdf22a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


## Building Fairseq

In [2]:
!git clone https://github.com/pytorch/fairseq.git
import os
os.chdir("fairseq/")
!pip install -r requirements.txt
%run -i 'setup.py' build develop

Cloning into 'fairseq'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 3274 (delta 93), reused 96 (delta 69), pack-reused 3093
Receiving objects: 100% (3274/3274), 3.27 MiB | 9.39 MiB/s, done.
Resolving deltas: 100% (2353/2353), done.
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/scripts
copying scripts/build_sym_alignment.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/average_checkpoints.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/__init__.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/read_binarized.py -> build/lib.linux-x86_64-3.6/scripts
creating build/lib.linux-x86_64-3.6/fairseq
copying fairseq/sequence_generator.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/bleu.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/tokenizer.py -> build/lib.linux-x86_64-3.

## Model for Ts


### Data

In [3]:
!git clone https://github.com/LauraMartinus/ukuxhumana.git

Cloning into 'ukuxhumana'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 1341 (delta 37), reused 29 (delta 12), pack-reused 1266
Receiving objects: 100% (1341/1341), 387.04 MiB | 21.66 MiB/s, done.
Resolving deltas: 100% (644/644), done.
Checking out files: 100% (303/303), done.


### Subword

In [4]:
!git clone https://github.com/rsennrich/subword-nmt

Cloning into 'subword-nmt'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 485 (delta 0), reused 1 (delta 0), pack-reused 481
Receiving objects: 100% (485/485), 205.64 KiB | 1.09 MiB/s, done.
Resolving deltas: 100% (287/287), done.


In [5]:
#os.chdir('../')
!ls

build		      fairseq		multiprocessing_train.py  scripts
CONTRIBUTING.md       fairseq.egg-info	PATENTS			  setup.py
distributed_train.py  fairseq.gif	preprocess.py		  subword-nmt
docs		      generate.py	README.md		  tests
eval_lm.py	      interactive.py	requirements.txt	  train.py
examples	      LICENSE		score.py		  ukuxhumana


## Study


**4k**

In [6]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 4000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:351: ResourceWarning: unclosed file <_io.TextIOWrapper name='ents.codes' mode='r' encoding='UTF-8'>
  args.codes = codecs.open(args.codes.name, encoding='utf-8')
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Pleas

**8k**

In [9]:
#os.chdir('../')
!ls

build		      examples		multiprocessing_train.py  setup.py
checkpoint	      fairseq		PATENTS			  subword-nmt
CONTRIBUTING.md       fairseq.egg-info	preprocess.py		  tests
data-bin	      fairseq.gif	README.md		  train.py
distributed_train.py  generate.py	requirements.txt	  ukuxhumana
docs		      interactive.py	score.py
eval_lm.py	      LICENSE		scripts


In [6]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 8000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:351: ResourceWarning: unclosed file <_io.TextIOWrapper name='ents.codes' mode='r' encoding='UTF-8'>
  args.codes = codecs.open(args.codes.name, encoding='utf-8')
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Pleas

**12k**

In [6]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 12000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:351: ResourceWarning: unclosed file <_io.TextIOWrapper name='ents.codes' mode='r' encoding='UTF-8'>
  args.codes = codecs.open(args.codes.name, encoding='utf-8')
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Pleas

**16k**

In [0]:
os.chdir('content/')
!ls

combine.txt  ennso.codes  fairseq  sample_data


In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 16000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

**20k**

In [0]:
!ls

build		      examples		multiprocessing_train.py  setup.py
checkpoint	      fairseq		PATENTS			  subword-nmt
CONTRIBUTING.md       fairseq.egg-info	preprocess.py		  tests
data-bin	      fairseq.gif	README.md		  train.py
distributed_train.py  generate.py	requirements.txt	  ukuxhumana
docs		      interactive.py	score.py
eval_lm.py	      LICENSE		scripts


In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 20000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

**24k**

In [0]:
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 24000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

**28k**

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 28000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

**32k**

In [0]:
!ls

build		      examples		multiprocessing_train.py  setup.py
checkpoint	      fairseq		PATENTS			  subword-nmt
CONTRIBUTING.md       fairseq.egg-info	preprocess.py		  tests
data-bin	      fairseq.gif	README.md		  train.py
distributed_train.py  generate.py	requirements.txt	  ukuxhumana
docs		      interactive.py	score.py
eval_lm.py	      LICENSE		scripts


In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 32000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

**36k**

In [0]:
#os.chdir('fairseq/')
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 36000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

**40k**

In [0]:
!ls

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 40000 <combine.txt> ents.codes

!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang ts --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang ts

## Results

*   40k: 
*   36k: 
*   32k: 
*   28k:
*   24k: 
*   20k: 
*   16k: 
*   12k: 35.75
*     8k: 34.94
*     4k: 33.21

